<a href="https://colab.research.google.com/github/vpysmennyi/machinelearning-learning/blob/main/Hometask7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

import torch
from datetime import datetime
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision.datasets import STL10
from torch.utils.data import DataLoader
from torchvision.utils import save_image


class Print(nn.Module):
    def __init__(self):
        super(Print, self).__init__()

    def forward(self, x):
        print(x.shape)
        return x


class Logger():
    def __init__(self, file = './CNN_execution.log'):
        self.logf = open(file, 'w')

    def write(self, text):
        self.logf.write(self.sysdate() + ' : ' + text + '\n')
        self.logf.flush()

    def close(self):
        self.logf.close()

    def sysdate(self):
        return datetime.now().strftime('%y%m%d %H:%M:%S')


class DAE(nn.Module):
    def __init__(self, pretrain=False):
        super(DAE, self).__init__()

        self.pretrain = pretrain

        self.enc = nn.Sequential(
            # 3x96x96
            self.convBlock(3, 64, kernel=(3, 3), padding=1),
            nn.MaxPool2d(3, 3),
            # 64x32x32
            self.convBlock(64, 64, (3, 3), 1),
            self.convBlock(64, 32, (3, 3), 1),
            self.convBlock(32, 16, (3, 3), 1),
            # 16x32x32
            nn.MaxPool2d(4, 4),
            # 16x8x8

            self.convBlock(16, 8, (3, 3), 1)

            # 8x8x8
        )

        self.dec = nn.Sequential(
            self.deconvBlock(8, 16, (3, 3), 1),
            nn.UpsamplingBilinear2d(scale_factor=4),

            self.deconvBlock(16, 32, (3, 3), 1),
            self.deconvBlock(32, 64, (3, 3), 1),
            self.deconvBlock(64, 64, (3, 3), 1),

            nn.UpsamplingBilinear2d(scale_factor=3),
            self.deconvBlock(64, 3, (3, 3), 1)
        )

        self.cl = nn.Sequential(nn.Linear(512, 256),
                           nn.ReLU(),
                           nn.BatchNorm1d(256),
                           nn.Linear(256,128),
                           nn.ReLU(),
                           nn.BatchNorm1d(128),
                           nn.Linear(128,10)
                           )

        if pretrain:
            self.net = nn.Sequential(self.enc, self.dec)
        else:
            self.net = nn.Sequential(self.enc, nn.Flatten(), self.cl)


    def convBlock(self, input, output, kernel, padding):
        return nn.Sequential(
            nn.Conv2d(input, output, kernel_size=kernel, padding=padding),
            nn.ReLU(),
            nn.BatchNorm2d(output))

    def deconvBlock(self, input, output, kernel, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(input, output, kernel_size=kernel, padding=padding),
            nn.ReLU(),
            nn.BatchNorm2d(output))
        
    def init_cl_weights(self, m):
         if type(m) == nn.Linear:
            nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
            m.bias.data.zero_()

    def forward(self, x):
        return self.net(x)


def train(dataset, network, loss_fn, opt):
    e_acc, e_loss = [], []

    network.train()
    for (X, Y) in dataset:
        opt.zero_grad()

        Yp = network(X)
        loss = loss_fn(Yp, Y)

        loss.backward()
        opt.step()

        acc = (Yp.argmax(1) == Y).sum() / len(Y)

        e_loss.append(loss.detach().cpu())
        if not network.pretrain:
            e_acc.append(acc)

    e_loss = torch.stack(e_loss).mean()

    if not network.pretrain:
        e_acc = torch.stack(e_acc).mean()
    
    return e_acc, e_loss

def validate(dataset, network, loss_fn):
    val_loss, val_acc = [],[]

    network.eval()
    for (X, Y) in dataset:
        Yp = network(X)

        loss = loss_fn(Yp, Y)
        acc = (Yp.argmax(1) == Y).sum() / len(Y)

        val_acc.append(acc)
        val_loss.append(loss.detach().cpu())

    val_loss = torch.stack(val_loss).mean()
    val_acc = torch.stack(val_acc).mean()

    return val_loss, val_acc

batch_size = 32
lr = 1e-3
pretrain_epochs = 10
num_epochs = 40
noise_factor = 0.3

train_ds = STL10('.', split='train', folds=None, transform=transforms.ToTensor(), download=True)
X = torch.stack([train_ds[i][0] for i in range(len(train_ds))], 1).reshape(3, -1)
stl10_mean, stl10_std = X.mean(1), X.std(1)
print(stl10_mean, stl10_std)

transform = transforms.Compose([transforms.RandomCrop(size=(96, 96), padding=3),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize(stl10_mean, stl10_std),
                                ])

val_transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(stl10_mean, stl10_std)
                                    ])

pretrain_ds = STL10('.', split='unlabeled', folds=None, transform=transforms.ToTensor(), download=True)
train_ds = STL10('.', split='train', folds=None, transform=transform, download=True)
val_ds = STL10('.', split='test', transform=val_transform, download=True)

pretrain_dl = DataLoader(pretrain_ds, batch_size=batch_size, shuffle=True)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=True)

log = Logger()

network = DAE(pretrain=True)

loss_fn = nn.MSELoss()
opt = optim.Adam(network.parameters(), lr=lr)

# pre-training
torch.cuda.init()

for epoch in range(pretrain_epochs):
    epoch_loss = []

    network.train()
    for batch in pretrain_dl:
        img, cl = batch

        noizy_img = img + noise_factor * torch.randn(img.shape)

        noizy_img = np.clip(noizy_img, 0, 1)
        opt.zero_grad()

        output = network(noizy_img)

        loss = loss_fn(output, noizy_img)

        loss.backward()
        opt.step()

        epoch_loss.append(loss.cpu().detach())

    epoch_loss = torch.stack(epoch_loss).mean()
    print(f' TRAIN | Epoch {epoch} - Loss: {epoch_loss:.6f}')


    torch.save(network.state_dict(), './drive/MyDrive/pretrain.pt')

nnt = DAE(pretrain=False)

loss_fn = nn.CrossEntropyLoss()
trainOpt = optim.Adam(nnt.parameters(), lr=lr)

nnt.load_state_dict(torch.load('./drive/MyDrive/pretrain.pt'), strict=False)

for epoch in range(num_epochs):
    e_acc, e_loss = train(train_dl,nnt,loss_fn,trainOpt)
    
    log.write(f' TRAIN | Epoch {epoch} - Loss: {e_loss:.6f} | Acc: {e_acc:.4f}')
    print(f' TRAIN | Epoch {epoch} - Loss: {e_loss:.6f} | Acc: {e_acc:.4f}')


    val_loss, val_acc = validate(val_dl,nnt,loss_fn)
    
    log.write(f' VAL   | Epoch {epoch} - Loss: {val_loss:.6f} | Acc: {val_acc:.4f}')
    print(f' VAL   | Epoch {epoch} - Loss: {val_loss:.6f} | Acc: {val_acc:.4f}')

log.close()

Files already downloaded and verified
tensor([0.4467, 0.4398, 0.4066]) tensor([0.2603, 0.2566, 0.2713])
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
 TRAIN | Epoch 0 - Loss: 1.640294 | Acc: 0.3915
 VAL   | Epoch 0 - Loss: 1.377521 | Acc: 0.4890
 TRAIN | Epoch 1 - Loss: 1.328106 | Acc: 0.5064
 VAL   | Epoch 1 - Loss: 1.321580 | Acc: 0.5215
 TRAIN | Epoch 2 - Loss: 1.207790 | Acc: 0.5611
 VAL   | Epoch 2 - Loss: 1.187133 | Acc: 0.5729
 TRAIN | Epoch 3 - Loss: 1.119371 | Acc: 0.5862
 VAL   | Epoch 3 - Loss: 1.187411 | Acc: 0.5685
 TRAIN | Epoch 4 - Loss: 1.060973 | Acc: 0.6186
 VAL   | Epoch 4 - Loss: 1.147045 | Acc: 0.5789
 TRAIN | Epoch 5 - Loss: 1.036159 | Acc: 0.6304
 VAL   | Epoch 5 - Loss: 1.300034 | Acc: 0.5700
 TRAIN | Epoch 6 - Loss: 1.003615 | Acc: 0.6320
 VAL   | Epoch 6 - Loss: 1.096066 | Acc: 0.6015
 TRAIN | Epoch 7 - Loss: 0.936484 | Acc: 0.6606
 VAL   | Epoch 7 - Loss: 1.096683 | Acc: 0.6074
 TRAIN | Epoch

In [ ]:
 # NO pretraining
 TRAIN | Epoch 0 - Loss: 1.993355 | Acc: 0.3107
 VAL   | Epoch 0 - Loss: 1.561598 | Acc: 0.4215
 TRAIN | Epoch 1 - Loss: 1.541898 | Acc: 0.4417
 VAL   | Epoch 1 - Loss: 1.374716 | Acc: 0.4886
 TRAIN | Epoch 2 - Loss: 1.365247 | Acc: 0.5004
 VAL   | Epoch 2 - Loss: 1.319850 | Acc: 0.5107
 TRAIN | Epoch 3 - Loss: 1.303779 | Acc: 0.5198
 VAL   | Epoch 3 - Loss: 1.335786 | Acc: 0.5071
 TRAIN | Epoch 4 - Loss: 1.226758 | Acc: 0.5509
 VAL   | Epoch 4 - Loss: 1.214242 | Acc: 0.5595
 TRAIN | Epoch 5 - Loss: 1.136996 | Acc: 0.5807

In [ ]:
#pretraining itself

 TRAIN | Epoch 0 - Loss: 0.152718
 TRAIN | Epoch 1 - Loss: 0.067189
 TRAIN | Epoch 2 - Loss: 0.066569
 TRAIN | Epoch 3 - Loss: 0.066246

In [ ]:
#classification with pretraining
 TRAIN | Epoch 0 - Loss: 1.640294 | Acc: 0.3915
 VAL   | Epoch 0 - Loss: 1.377521 | Acc: 0.4890
 TRAIN | Epoch 1 - Loss: 1.328106 | Acc: 0.5064
 VAL   | Epoch 1 - Loss: 1.321580 | Acc: 0.5215
 TRAIN | Epoch 2 - Loss: 1.207790 | Acc: 0.5611
 VAL   | Epoch 2 - Loss: 1.187133 | Acc: 0.5729
 TRAIN | Epoch 3 - Loss: 1.119371 | Acc: 0.5862
 VAL   | Epoch 3 - Loss: 1.187411 | Acc: 0.5685
 TRAIN | Epoch 4 - Loss: 1.060973 | Acc: 0.6186
 VAL   | Epoch 4 - Loss: 1.147045 | Acc: 0.5789
 TRAIN | Epoch 5 - Loss: 1.036159 | Acc: 0.6304
 VAL   | Epoch 5 - Loss: 1.300034 | Acc: 0.5700
 TRAIN | Epoch 6 - Loss: 1.003615 | Acc: 0.6320
 VAL   | Epoch 6 - Loss: 1.096066 | Acc: 0.6015
 TRAIN | Epoch 7 - Loss: 0.936484 | Acc: 0.6606
 VAL   | Epoch 7 - Loss: 1.096683 | Acc: 0.6074
 TRAIN | Epoch 8 - Loss: 0.898440 | Acc: 0.6835
 VAL   | Epoch 8 - Loss: 1.234113 | Acc: 0.6064
 TRAIN | Epoch 9 - Loss: 0.862782 | Acc: 0.6919
 VAL   | Epoch 9 - Loss: 1.071374 | Acc: 0.6256
 TRAIN | Epoch 10 - Loss: 0.844233 | Acc: 0.6875
 VAL   | Epoch 10 - Loss: 1.275288 | Acc: 0.5914
 TRAIN | Epoch 11 - Loss: 0.789699 | Acc: 0.7116
 VAL   | Epoch 11 - Loss: 1.078518 | Acc: 0.6256
 TRAIN | Epoch 12 - Loss: 0.762762 | Acc: 0.7257
 VAL   | Epoch 12 - Loss: 1.035229 | Acc: 0.6420
 TRAIN | Epoch 13 - Loss: 0.740810 | Acc: 0.7321
 VAL   | Epoch 13 - Loss: 1.133314 | Acc: 0.6183
 TRAIN | Epoch 14 - Loss: 0.733109 | Acc: 0.7317
 VAL   | Epoch 14 - Loss: 1.136526 | Acc: 0.6406
 TRAIN | Epoch 15 - Loss: 0.686323 | Acc: 0.7484
 VAL   | Epoch 15 - Loss: 1.281064 | Acc: 0.6265
 TRAIN | Epoch 16 - Loss: 0.656867 | Acc: 0.7619
 VAL   | Epoch 16 - Loss: 1.245477 | Acc: 0.6267